<a href="https://colab.research.google.com/github/tonystz/gitpod/blob/main/test_pycuda_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to CUDA and PyCUDA

In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:
!nvidia-smi

Tue Mar 14 11:40:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    28W /  70W |    103MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pycuda
import pycuda.driver as drv
drv.init()

print('CUDA device query (PyCUDA version) \n')

print('Detected {} CUDA Capable device(s) \n'.format(drv.Device.count()))

for i in range(drv.Device.count()):
    
    gpu_device = drv.Device(i)
    print('Device {}: {}'.format( i, gpu_device.name() )) 
    compute_capability = float( '%d.%d' % gpu_device.compute_capability() )
    print('\t Compute Capability: {}'.format(compute_capability))
    print('\t Total Memory: {} megabytes'.format(gpu_device.total_memory()//(1024**2)))


    # The following will give us all remaining device attributes as seen 
    # in the original deviceQuery.
    # We set up a dictionary as such so that we can easily index
    # the values using a string descriptor.
    
    device_attributes_tuples = iter(gpu_device.get_attributes().items()) 
    device_attributes = {}

        
    for k, v in device_attributes_tuples:
        device_attributes[str(k)] = v
        # print(f'{k}->{v}')
    # continue
    num_mp = device_attributes['MULTIPROCESSOR_COUNT']
    
    # Cores per multiprocessor is not reported by the GPU!  
    # We must use a lookup table based on compute capability.
    # See the following:
    # http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities
    
    cuda_cores_per_mp = { 5.0 : 128, 5.1 : 128, 5.2 : 128, 6.0 : 64, 6.1 : 128, 7.5 : 128}[compute_capability]
    
    print('\t ({}) Multiprocessors, ({}) CUDA Cores / Multiprocessor: {} CUDA Cores'.format(num_mp, cuda_cores_per_mp, num_mp*cuda_cores_per_mp))
    
    device_attributes.pop('MULTIPROCESSOR_COUNT')
    
    for k in list(device_attributes.keys()):
        print('\t {}: {}'.format(k, device_attributes[k]))


CUDA device query (PyCUDA version) 

Detected 1 CUDA Capable device(s) 

Device 0: Tesla T4
	 Compute Capability: 7.5
	 Total Memory: 15101 megabytes
	 (40) Multiprocessors, (128) CUDA Cores / Multiprocessor: 5120 CUDA Cores
	 ASYNC_ENGINE_COUNT: 3
	 CAN_MAP_HOST_MEMORY: 1
	 CAN_USE_HOST_POINTER_FOR_REGISTERED_MEM: 1
	 CLOCK_RATE: 1590000
	 COMPUTE_CAPABILITY_MAJOR: 7
	 COMPUTE_CAPABILITY_MINOR: 5
	 COMPUTE_MODE: DEFAULT
	 COMPUTE_PREEMPTION_SUPPORTED: 1
	 CONCURRENT_KERNELS: 1
	 CONCURRENT_MANAGED_ACCESS: 1
	 DIRECT_MANAGED_MEM_ACCESS_FROM_HOST: 0
	 ECC_ENABLED: 1
	 GENERIC_COMPRESSION_SUPPORTED: 0
	 GLOBAL_L1_CACHE_SUPPORTED: 1
	 GLOBAL_MEMORY_BUS_WIDTH: 256
	 GPU_OVERLAP: 1
	 HANDLE_TYPE_POSIX_FILE_DESCRIPTOR_SUPPORTED: 1
	 HANDLE_TYPE_WIN32_HANDLE_SUPPORTED: 0
	 HANDLE_TYPE_WIN32_KMT_HANDLE_SUPPORTED: 0
	 HOST_NATIVE_ATOMIC_SUPPORTED: 0
	 INTEGRATED: 0
	 KERNEL_EXEC_TIMEOUT: 0
	 L2_CACHE_SIZE: 4194304
	 LOCAL_L1_CACHE_SUPPORTED: 1
	 MANAGED_MEMORY: 1
	 MAXIMUM_SURFACE1D_LAYERED_LAY

In [ ]:
import numpy

a = numpy.array([[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]], dtype=numpy.float32)

data=numpy.zeros((64, 64), dtype='int')
print(data)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
a_gpu = cuda.mem_alloc(a.nbytes)

In [ ]:
cuda.memcpy_htod(a_gpu, a)


##pycuda index test

In [ ]:
%%writefile a.py
#!python 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np

mod = SourceModule("""
    #include <stdio.h>

    __global__ void say_hi(int *out_gpu)
    { 
      int idx = threadIdx.x + blockIdx.x * blockDim.x;
      int idy = threadIdx.y + blockIdx.y * blockDim.y;
      __shared__ double out_buf[1024];
      int l_count = 0;
      __shared__ int g_count;
      l_count +=1;

      printf("I am threadIdx:[%d[%d]  of block:[%d][%d] block size:[%d][%d][%d]  grid size:[%d][%d]:\\n", threadIdx.x, threadIdx.y, blockIdx.x,blockIdx.y,blockDim.x,blockDim.y,blockDim.z, gridDim.x,gridDim.y);
      

      out_buf[idx]=idy;

      __syncthreads();
      g_count +=1;
      //how to pass out the modified array data
      __syncthreads();
      out_gpu[idx]=2;
      __syncthreads();
      printf("thread id: [%d][%d]   -> l_count=%d ,g_count=%d *out_gpu=%d\\n",idx,idy,l_count,g_count,*out_gpu);
    }
    """)

func = mod.get_function("say_hi")


data=np.zeros(64, dtype='int')
print('shape:',data.shape)
data_gpu = gpuarray.to_gpu(data)
out_gpu = gpuarray.empty_like(data_gpu)
func(out_gpu,block=(3,1,1),grid=(1,1,1))
print('modify pointer array:',out_gpu.get())


Overwriting a.py


In [ ]:
!python a.py

shape: (64,)
I am threadIdx:[0[0]  of block:[0][0] block size:[3][1][1]  grid size:[1][1]:
I am threadIdx:[1[0]  of block:[0][0] block size:[3][1][1]  grid size:[1][1]:
I am threadIdx:[2[0]  of block:[0][0] block size:[3][1][1]  grid size:[1][1]:
thread id: [0][0]   -> l_count=1 ,g_count=0 *out_gpu=2
thread id: [1][0]   -> l_count=1 ,g_count=0 *out_gpu=2
thread id: [2][0]   -> l_count=1 ,g_count=0 *out_gpu=2
modify pointer array: [8589934594          2          0          0          0          0
          0          0          0          0          0          0
          0          0          0          0          0          0
          0          0          0          0          0          0
          0          0          0          0          0          0
          0          0          0          0          0          0
          0          0          0          0          0          0
          0          0          0          0          0          0
          0          0        

In [122]:
%%writefile sh.py
#!python 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np

mod = SourceModule("""
    #include <stdio.h>

    __global__ void say_hi(int *out_gpu)
    { 
      int idx = threadIdx.x + blockIdx.x * blockDim.x;
      int idy = threadIdx.y + blockIdx.y * blockDim.y;

      printf("I am threadIdx:[%d][%d]  of block:[%d][%d] block size:[%d][%d][%d]  grid size:[%d][%d]  idx=%d, idy=%d\\n", threadIdx.x, threadIdx.y, blockIdx.x,blockIdx.y,blockDim.x,blockDim.y,blockDim.z, gridDim.x,gridDim.y,idx,idy);
      //for(int i<0;i<3;i++){
          printf("%d\\n",out_gpu[idx*3+idy]);
          //printf("%d\\n",out_gpu[idx]);
      //}
     // __syncthreads();
  
    }
    """)

func = mod.get_function("say_hi")


data=np.array([21,2,3,38,9,10], dtype=np.int32)
data=data.reshape(2,3)
print('shape:',data.shape,data)
data_gpu = gpuarray.to_gpu(data)
func(data_gpu,block=(2,3,1),grid=(1,1,1))
#print('modify pointer array:',data_gpu.get())

Overwriting sh.py


In [123]:
!python sh.py

shape: (2, 3) [[21  2  3]
 [38  9 10]]
I am threadIdx:[0][0]  of block:[0][0] block size:[2][3][1]  grid size:[1][1]  idx=0, idy=0
I am threadIdx:[1][0]  of block:[0][0] block size:[2][3][1]  grid size:[1][1]  idx=1, idy=0
I am threadIdx:[0][1]  of block:[0][0] block size:[2][3][1]  grid size:[1][1]  idx=0, idy=1
I am threadIdx:[1][1]  of block:[0][0] block size:[2][3][1]  grid size:[1][1]  idx=1, idy=1
I am threadIdx:[0][2]  of block:[0][0] block size:[2][3][1]  grid size:[1][1]  idx=0, idy=2
I am threadIdx:[1][2]  of block:[0][0] block size:[2][3][1]  grid size:[1][1]  idx=1, idy=2
21
38
2
9
3
10


In [97]:
import numpy as np
data=np.array([21,2,3,38,9,10], dtype=np.int32)
print(data.reshape(2,3))
print(data)

[[21  2  3]
 [38  9 10]]
[21  2  3 38  9 10]


### Run naive_prefix.py

In [ ]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda import gpuarray
from pycuda.compiler import SourceModule
from time import time
# this is a naive parallel prefix-sum kernel that uses shared memory
naive_ker = SourceModule("""
__global__ void naive_prefix(double *vec, double *out)
{
     __shared__ double sum_buf[1024];     
     int tid = threadIdx.x;     
     sum_buf[tid] = vec[tid];
     
     // begin parallel prefix sum algorithm
     
     int iter = 1;
     for (int i=0; i < 10; i++)
     {
         __syncthreads();
         if (tid >= iter )
         {
             sum_buf[tid] = sum_buf[tid] + sum_buf[tid - iter];            
         }
         
         iter *= 2;
     }
         
    __syncthreads();
    out[tid] = sum_buf[tid];
    __syncthreads();
        
}
""")
naive_gpu = naive_ker.get_function("naive_prefix")
    


if __name__ == '__main__':
    
    
    testvec = np.random.randn(1024).astype(np.float64)
    testvec_gpu = gpuarray.to_gpu(testvec)
    
    outvec_gpu = gpuarray.empty_like(testvec_gpu)

    naive_gpu( testvec_gpu , outvec_gpu, block=(1024,1,1), grid=(1,1,1))
    
    total_sum = sum( testvec)
    total_sum_gpu = outvec_gpu[-1].get()
    
    print('outvec_gpu:',outvec_gpu)
    print("Does our kernel work correctly? : {}".format(np.allclose(total_sum_gpu , total_sum) ))


outvec_gpu: [ -1.20500879  -1.6132585   -2.71519557 ... -57.63849801 -58.57298875
 -59.34142221]
Does our kernel work correctly? : True


### hello world
https://documen.tician.de/pycuda/tutorial.html

[[1 0 0]
 [1 1 0]
 [0 0 1]]


### print input

In [ ]:
%%writefile i.py
#!python 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np

mod = SourceModule("""
    #include <stdio.h>

    __global__ void say_hi(int *out_gpu)
    { 
      int idx = threadIdx.x + blockIdx.x * blockDim.x;
     

      printf("index: threadIdx.x =%d blockIdx.x =%d blockDim.x %d  \\n", threadIdx.x , blockIdx.x , blockDim.x);
      printf("thread id: [%d]  intput out_gpu=%d \\n",idx,out_gpu[idx]);
      
      out_gpu[idx] *= 2;
      printf("thread id 2: [%d]  out_gpu=%d \\n",idx,out_gpu[idx]);

    }
    """)

func = mod.get_function("say_hi")


t= np.array([1,2,3,4], dtype=np.int32)
print('shape:',t.shape,t.size)
t_gpu = gpuarray.to_gpu(t)
func(t_gpu,block=(t.size,1,1),grid=(1,1,1))
print(t_gpu.get())

Overwriting i.py


In [ ]:
!python i.py

shape: (4,) 4
index: threadIdx.x =0 blockIdx.x =0 blockDim.x 4  
index: threadIdx.x =1 blockIdx.x =0 blockDim.x 4  
index: threadIdx.x =2 blockIdx.x =0 blockDim.x 4  
index: threadIdx.x =3 blockIdx.x =0 blockDim.x 4  
thread id: [0]  intput out_gpu=1 
thread id: [1]  intput out_gpu=2 
thread id: [2]  intput out_gpu=3 
thread id: [3]  intput out_gpu=4 
thread id 2: [0]  out_gpu=2 
thread id 2: [1]  out_gpu=4 
thread id 2: [2]  out_gpu=6 
thread id 2: [3]  out_gpu=8 
[2 4 6 8]


### pass string to kernel

In [ ]:
import numpy as np

s= np.array(['abc','hello,world'], dtype=object)
print(s)

['abc' 'hello,world']


In [ ]:
%%writefile s.py
#!python 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np

mod = SourceModule("""
    #include <stdio.h>

    __global__ void say_hi(int *out_gpu)
    { 
      int idx = threadIdx.x + blockIdx.x * blockDim.x;
      printf("thread id: [%d]  %d\\n",idx,out_gpu);
      //print c-str
      for (int i=0;i<5;i++){
        printf("%c",out_gpu[0]);
      }
      printf(" >end\\n");

    }
    """)

func = mod.get_function("say_hi")


s= np.array(['abcdf'], dtype=object)
print('shape:',s.shape)
print(s,s.data)
s_gpu = gpuarray.to_gpu(s)
func(s_gpu,block=(s.size,1,1),grid=(1,1,1))


Overwriting s.py


In [ ]:
!python s.py

/content/s.py:8: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(8): warning #181-D: argument is incompatible with corresponding format string conversion


  mod = SourceModule("""
shape: (1,)
['abcdf'] <memory at 0x7f66026f9640>
thread id: [0]  0
����� >end


In [ ]:
mod2 = SourceModule("""
  __global__ void add2(float *a, float *b)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] += b[idx];
  }
  """)

In [ ]:
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)


In [ ]:
func = mod2.get_function("add2")
func(b_gpu,c_gpu, block=(4,4,1))

In [ ]:
added = numpy.empty_like(b)
cuda.memcpy_dtoh(added, b_gpu)
print(added)
print(b)
print(c)

# Exercises

1. Write a cuda kernel to find the elementwise square of a matrix
2. Write a cuda kernel to find a matrix, which when added to the given matrix results in every element being equal to zero
3. Write a cuda kernel to multiply two matrices:
    1. Assume square matrices, with dimensions < 1024
    2. Assume square matrices, with dimensions > 1024
    3. Assume non-square matrices, with dimensions > 1024

In [ ]:
1. 